# Libraries, organizing your code

The classic HDL way is to write a simple file with one `@block` decorated hardware unit and import it as a module. However this is not sufficient in some cases when full portability or extended configureability is desired.

Also, one might use a library of existing, external VHDL or Verilog code.

## User libraries

When starting a new set of hardware elements from scratch, the best approach is to create `@block_component` objects inside a module class.

In particular, when a hardware description should simulate and synthesize, it is desirable to have a base class and use inheritance to implement more
features or architecture-specific variants.

* [Library concepts: Library classes](../notebooks/library_concepts.ipynb#Library-classes)


## External HDL libraries

External HDL files are technically treated as black boxes and are referenced as a stub within the target design.

Some targets might support file-only blackboxes. The `@blackbox` decorator itself just reserves the stub, but does not provide means to pull the file.

For a library, this is inconvenient, we'd rather see that automated and explicitely mapping to a target language.
The `@blackbox_verilog` decorator takes a `path_prefix` to the file whose filename is automatically constructed from the function name. Likewise, there is a `@blackbox_vhdl` decorator.

In [1]:
from cyhdl import *

Bool = Signal.Type(bool)
Byte = Signal.Type(intbv()[8:])

from myirl.blackbox_ext import *

@blackbox_verilog(path_prefix="./verilog")
def debug(clk : ClkSignal, en : Bool, i_data : Byte, o_data : Byte):
    pass

Verify the reference to the external Verilog file:

In [2]:
debug.get_sources()

['./verilog/debug.v']

## Primitives

**Warning** Synthesizer specific

Some synthesis packages offer a library of built-in primitives that can be instanced directly from your code a a component.

Even though they can be considered white or gray boxes, they are treated as a special `@blackbox` component on the IRL kernel level, because they may not provide a native Python description that may possibly simulate. In most cases, external vendor specific simulation models are provided.

For **yosys** and a few supported architectures, wrappers are provided for the openly available simulation models.


### Example

In [3]:
from yosys.builtins import builtins

@block
def unit(clk : ClkSignal, data_in : Signal, data_out : Signal.Output):
    
    # tmp = data_out.clone()
    
    inst_dff = builtins.Dff(CLK = clk,
                            D = data_in,
                            Q = data_out,
                            CLK_POLARITY = True,
                            WIDTH = data_in.size()
                           )
    
    return [ inst_dff ]

In [4]:
a, b = [ Signal(intbv()[12:], name = n) for n in "ab" ]
clk = ClkSignal()

u = unit(clk, a, b)

from myirl.targets.pyosys import RTLIL
RTL = RTLIL("default")

objs = u.elab(RTL)
d = objs[0]

DEBUG INIT IN CONTEXT Dff (LIB: _Builtins '_yosys_builtins_')
 [_builtin_method 'Dff/Dff'] blackbox not returning instances 
 Adding module with name `unit` 
 PARAM CLK_POLARITY --> True 
 PARAM WIDTH --> 12 
 FINALIZE implementation `unit` of `unit` 


/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/components.py:205: UserWarning: @component `unit` interface :Output data_out not driven
  base.warnings.warn(self._err("Output %s not driven" % \


In [5]:
d.display_rtl(selection = unit, fmt='dot')

In [6]:
from yosys import display
display.display_dot(d.name)

## General vendor blackbox issues

If a FPGA vendor's blackbox is to be instanced directly, it may be important to sort out at an early stage if:

* It is supplied with a binding for synthesis
* If is supplied with a non-encrypted simulation file for the OpenSource simulators GHDL or Verilog

In the case of encrypted simulation files, the only legal way to output towards simulation is:

* Choose the provided simulator for your FPGA target
* Output your design to Verilog

### CXXRTL compatibility

From the vendor's perspective, the it is to be kept in mind that black box models must be synthesizeable using yosys and may not contain specific timing information. Therefore, all sorts of clock generators such as PLLs can not be compiled into a simulation.